### 7.1 Word Embedding 

In [ ]:
# - WE , kelimeleri sayisal vektörlerle ifade eden bir yöntemdir. 
# - Bu temsiller, kelimeler arasındaki anlamsal ilişkileri öğrenmeyi saglar.
# - Aynı anlamda veya benzer anlamda kullanılan kelimeler vektor uzayında birbirine yakın olur.

# CBOW = baglam kelimelere bakrak kelime tahmin etme
# Skip-gram = CBOW tersine çalısıyor yani kelimeye bkılarak baglam kelimeleri tahmin ediyor.

### 7.2 RNN 

In [ ]:
# - Zaman serisi verisi 
# - Sekans Verisi ( Sıralı Veri )

# Rnn' nin özellikleri 
    # - Zaman Boyutunda tekrar 
    # Sekans Verisi için uygun

# RNN ile yapılan uygulamalar; 
#     - Dil Modelleme 
#     - Konuşma Tanıma
#     - Makine Çevirisi
#     - Görüntü Tanıma    
#     - Metin üretimi
#     -Duygu analizi

#### 7.3 RNN 1 

In [ ]:
"""
Solve Classification problem (Sentiment Analysis in NLP) with RNN (Deep Learning based Language Model)

duygu analizi -> bir cumlenin etiketlenmesi (positive ve negative)
restaurant yorumlari degerlendirme
"""

# import libraries
import numpy as np
import pandas as pd

from gensim.models import Word2Vec # metin temsili

from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Embedding
from keras.preprocessing.text import Tokenizer

from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder

# dataset
data = {
    "text": [
        "Yemekler harikaydı, her şey taze ve lezzetliydi.",
        "Garson çok ilgisizdi, siparişimi unuttular.",
        "Tatlılar gerçekten çok güzeldi, bayıldım!",
        "Yemekler soğuktu ve tadı hiç hoş değildi.",
        "Atmosfer oldukça keyifliydi, tekrar geleceğim.",
        "Fiyatlar biraz yüksekti ama yemekler güzeldi.",
        "Servis kalitesi çok iyiydi, teşekkürler.",
        "Yemek çok geç geldi, sabrım kalmadı.",
        "Lezzetli bir akşam yemeği deneyimledik.",
        "Bu restoranı asla tavsiye etmem, kötüydü.",
        "Mekan çok hoştu, özellikle dekorasyonu.",
        "Yemekler beklediğimden çok daha kötüydü.",
        "Güzel bir akşam geçirdik, teşekkürler.",
        "Yemekler fazlasıyla tuzlu geldi, hiç beğenmedim.",
        "Kahvaltı muhteşemdi, her şeyi denemek istedim.",
        "Fiyatlar oldukça makuldü, çok memnun kaldım.",
        "Garsonlar çok yardımseverdi, teşekkürler.",
        "Yemekler güzel ama servis biraz yavaştı.",
        "Çocuklar için harika bir yer, çok eğlendiler.",
        "Bir daha asla gitmeyeceğim, kötü bir deneyim yaşadım.",
        "Mekanın atmosferi çok keyifliydi.",
        "Yemeklerin tadı harikaydı, özellikle deniz ürünleri.",
        "Şarap menüsü oldukça zengindi, beğendim.",
        "Yemekler sıcak servis edilmedi, hayal kırıklığıydı.",
        "Burgerleri gerçekten çok lezzetliydi.",
        "Tatlıların fiyatı biraz yüksekti ama lezzetliydi.",
        "Hizmet çok yavaştı ama yemekler fena değildi.",
        "Gerçekten güzel bir akşam yemeği deneyimi yaşadık.",
        "Sushi taze ve lezzetliydi, kesinlikle tavsiye ederim.",
        "Garsonlar çok nazik ve yardımseverdi.",
        "Hizmetin daha iyi olmasını beklerdim.",
        "Kahvaltı menüsü oldukça zengindi, çok beğendim.",
        "Yemekler çok lezzetliydi ama servis biraz yavaştı.",
        "Fiyatlar oldukça makuldü, bu kadar iyi hizmete.",
        "Mekan çok temizdi, bu benim için önemli.",
        "Tatlıların çok şekerli olduğunu düşündüm.",
        "Hizmet yavaş ama mekan güzeldi.",
        "Yemeklerin lezzeti harikaydı ama porsiyonlar küçük.",
        "Kendimi çok özel hissettim, teşekkürler.",
        "Güzel bir akşam yemeği, tekrar geleceğim.",
        "Çalışanlar çok güler yüzlüydü.",
        "Pasta çok güzeldi, özellikle çikolatalı.",
        "Biraz beklemek zorunda kaldık ama değdi.",
        "Sadece fiyatlar biraz yüksekti ama lezzet buna değer.",
        "Mekan oldukça kalabalıktı ama hizmet güzel.",
        "Garsonlar çok nazik ama biraz daha hızlı olabilirdi.",
        "Yemeklerin sunumu gerçekten etkileyiciydi.",
        "Yemekler çok lezzetliydi ama garsonlar nazik değildi.",
        "Çok güzel bir akşam yemeği deneyimi yaşadım.",
        "Pasta siparişi verdim ama çok uzun sürdü."
    ],
    "label": [
        "positive", "negative", "positive", "negative", "positive",
        "positive", "positive", "negative", "positive", "negative",
        "positive", "negative", "positive", "negative", "positive",
        "positive", "positive", "positive", "negative", "negative",
        "positive", "positive", "positive", "negative", "positive",
        "negative", "positive", "positive", "positive", "positive",
        "negative", "positive", "positive", "negative", "negative",
        "negative", "positive", "positive", "positive", "positive",
        "positive", "positive", "positive", "positive", "negative",
        "negative", "positive", "positive", "positive", "negative"

    ]
}
df = pd.DataFrame(data)

In [2]:
df.head(3)

,text,label
0,"Yemekler harikaydı, her şey taze ve lezzetliydi.",positive
1,"Garson çok ilgisizdi, siparişimi unuttular.",negative
2,"Tatlılar gerçekten çok güzeldi, bayıldım!",positive


#### 7.4 RNN 2

In [5]:
# metin temizleme ve preprocessing: tokenization, padding, label encoding, train test split

# tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
word_index = tokenizer.word_index

# padding process 
maxlen = max(len(seq) for seq in sequences)
X = pad_sequences(sequences, maxlen = maxlen)
print(X.shape)

# label encoding 
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df["label"])

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

(50, 8)


In [6]:
# metin temsili: word embedding: word2vec

sentences = [text.split() for text in df["text"]]
word2vec_model = Word2Vec(sentences, vector_size=50, window = 5, min_count=1) 

embedding_dim = 50
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]

#### 7.5 RNN 3

In [7]:
# modelling: build, train ve test rnn modeli 

# build model
model = Sequential()

# embedding
model.add(Embedding(input_dim = len(word_index) + 1, output_dim = embedding_dim, weights = [embedding_matrix], input_length=maxlen, trainable = False))

# RNN layer
model.add(SimpleRNN(50, return_sequences = False))

# output layer
model.add(Dense(1, activation="sigmoid"))

# compile model
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics=["accuracy"])

# train model
model.fit(X_train, y_train, epochs=10, batch_size = 2, validation_data=(X_test, y_test))

# evaluate rnn model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")



Epoch 1/10


20/20 [==============================] - 2s 18ms/step - loss: 0.6717 - accuracy: 0.6500 - val_loss: 0.7201 - val_accuracy: 0.5000
Epoch 2/10
20/20 [==============================] - 0s 4ms/step - loss: 0.6616 - accuracy: 0.7000 - val_loss: 0.7957 - val_accuracy: 0.5000
Epoch 3/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5934 - accuracy: 0.7000 - val_loss: 0.7425 - val_accuracy: 0.5000
Epoch 4/10
20/20 [==============================] - 0s 4ms/step - loss: 0.6045 - accuracy: 0.7000 - val_loss: 0.7602 - val_accuracy: 0.5000
Epoch 5/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5965 - accuracy: 0.7000 - val_loss: 0.7901 - val_accuracy: 0.5000
Epoch 6/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5856 - accuracy: 0.7000 - val_loss: 0.8081 - val_accuracy: 0.5000
Epoch 7/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5632 - accuracy: 0.7000 - val_loss: 0.7644 - val_accuracy: 0.5000
Epoch 8/10
20/2

In [8]:
# cumle siniflandirma calismasi
def classify_sentence(sentence):
    
    seq = tokenizer.texts_to_sequences([sentence])
    padded_seq = pad_sequences(seq, maxlen = maxlen) 
    
    prediction = model.predict(padded_seq)
    
    predicted_class = (prediction > 0.5).astype(int)
    label = "positive" if predicted_class[0][0] == 1 else "negative"
    
    return label

sentence = "Restaurant çok temizdi ve yemekler çok güzeldi"

result = classify_sentence(sentence)
print(f"Result: {result}")

1/1 [==============================] - 0s 278ms/step
Result: positive


### 7.6 LSTM ( Long Short-Term Memory )

![ScreenS/7.6_LSTM.PNG](ScreenS/7.6_LSTM.PNG)

![ScreenS/RNNvsLSTM.PNG](ScreenS/RNNvsLSTM.PNG)

#### 7.7 LSTM 1

In [8]:
"""
metin uretimi
lstm train with text data
text data = gpt ile olustur
"""

# import libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# egitim verisi chatgpt ile olustur
texts = [
    "Bugün hava çok güzel, dışarıda yürüyüş yapmayı düşünüyorum.",
    "Kitap okumak beni gerçekten mutlu ediyor.",
    "Kahve içmeden güne başlamak zor geliyor.",
    "Akşam yemeğinde pizza yemeyi planlıyorum.",
    "Sinemaya gitmek her zaman keyifli bir aktivite.",
    "Sabah koşusu bana enerji veriyor.",
    "Yeni bir dil öğrenmek beni heyecanlandırıyor.",
    "Hafta sonu arkadaşlarımla buluşacağım.",
    "Doğada vakit geçirmek beni rahatlatıyor.",
    "Müzik dinlemek beni motive ediyor.",
    "Yarın önemli bir toplantım var, çok heyecanlıyım.",
    "Yeni bir kitap aldım, bu hafta sonu okumayı planlıyorum.",
    "Akşam dışarıda yemek yemeyi düşünüyorum.",
    "Bu sabah yoga yaparak güne başladım.",
    "Sıcak bir çay içmek beni her zaman rahatlatır.",
    "Şehirde yeni bir restoran açılmış, gitmek istiyorum.",
    "Uzun zamandır bisiklete binmiyordum, bugün binmeyi planlıyorum.",
    "Bahçede çiçekler açmış, harika görünüyor.",
    "Bu hafta çok yoğundum, biraz dinlenmeye ihtiyacım var.",
    "Yeni bir diziye başladım, oldukça sürükleyici.",
    "Çalışmalarımda daha verimli olmak istiyorum.",
    "Bugün alışveriş yapmam gerekiyor, eksiklerim var.",
    "Sabahları yürüyüş yapmak benim için iyi bir alışkanlık oldu.",
    "Yeni tarifler denemek mutfakta zaman geçirmemi keyifli kılıyor.",
    "Uzun zamandır görmediğim bir arkadaşımla buluşacağım.",
    "Ders çalışırken sessiz bir ortamda olmak bana daha çok odaklanmamı sağlıyor.",
    "Bugün işlerimi erken bitirip biraz dinleneceğim.",
    "Hafta sonu piknik yapmayı düşünüyorum.",
    "Yaz tatili için plan yapmaya başladım.",
    "Bugün kendime biraz zaman ayırıp film izlemeyi düşünüyorum.",
    "Kış aylarını seviyorum, çünkü sıcak çikolata içmeyi çok seviyorum.",
    "Telefonumun şarjı bitmek üzere, hemen şarja takmam lazım.",
    "İşimle ilgili yeni projeler üzerinde çalışıyorum.",
    "Bugün spora gitmek beni gerçekten zorlayacak.",
    "Tiyatroya gitmeyi uzun zamandır planlıyordum, bu hafta gitmeyi düşünüyorum.",
    "Yeni müzikler keşfetmek her zaman ilgimi çekiyor.",
    "Yarın sabah erkenden uyanmam gerekiyor.",
    "Dışarıda çok güzel bir hava var, belki biraz bisiklet sürerim.",
    "Bahçeye birkaç yeni bitki ekmeyi düşünüyorum.",
    "Kütüphaneye gidip yeni kitaplar bakacağım.",
    "Evdeki işlerimi bitirip bir kahve molası vereceğim.",
    "Gelecek hafta için bir tatil planı yapıyorum.",
    "Bugün işyerinde toplantılar arka arkaya sıralanmış.",
    "Yeni bir hobi edinmek istiyorum, belki resim yapmayı deneyebilirim.",
    "Bu hafta spor salonuna düzenli olarak gitmeye karar verdim.",
    "Sabahları meditasyon yapmak beni çok rahatlatıyor.",
    "Ailecek bu akşam film gecesi yapacağız.",
    "Yaz mevsimi yaklaşırken, deniz tatili planları yapıyorum.",
    "Bugün işim erken biterse bir kitap okumayı planlıyorum.",
    "Yeni tarifler denemek mutfakta daha fazla zaman geçirmemi sağlıyor.",
    "Bugün alışveriş merkezine gidip ihtiyaçlarımı tamamlayacağım.",
    "Gelecek hafta arkadaşlarımla bir doğa yürüyüşüne çıkmayı düşünüyoruz.",
    "Yatmadan önce biraz müzik dinlemek rahatlamamı sağlıyor.",
    "Bugün biraz daha geç kalktım ve kendime vakit ayırdım.",
    "Yarın işim erken biterse yeni bir diziye başlayabilirim.",
    "Mutfakta yeni bir tatlı denemek istiyorum.",
    "Bu hafta sonu şehir dışında bir yere gitmeyi planlıyoruz.",
    "Yeni bir fotoğraf makinesi aldım, hafta sonu denemek için sabırsızlanıyorum.",
    "Sabahları güneşin doğuşunu izlemek bana enerji veriyor.",
    "İş yoğunluğu arasında kısa bir mola vermek bana iyi geliyor.",
    "Akşam yemeği için dışarıda bir yer arıyorum.",
    "Bugün havalar biraz soğudu, kalın giyinmem gerek.",
    "Yeni bir film izlemek için sinemaya gitmeyi düşünüyorum.",
    "Bu hafta sonu evde dinlenip enerji toplamayı planlıyorum.",
    "İşimle ilgili bir sunum hazırlamam gerekiyor.",
    "Yaz tatili için sahil kasabasına gitmek istiyorum.",
    "Dışarıda yağmur yağıyor, tam kitap okuma havası.",
    "Sabah kahvaltısında yeni bir şeyler denemeyi seviyorum.",
    "Bugün ofiste biraz yoğun bir gün geçirdim.",
    "Kış ayları yaklaşıyor, dolabımı yenilemem gerek.",
    "Yeni bir spor dalı denemek istiyorum, belki yoga.",
    "Hafta sonu için arkadaşlarımla bir etkinlik planlıyoruz.",
    "Bu akşam evde kendime biraz zaman ayırıp dinleneceğim.",
    "Yeni bir bilgisayar oyunu keşfettim, oldukça eğlenceli.",
    "Sabahları kahve içmeden kendime gelemiyorum.",
    "Yarın sabah erkenden uyanıp yürüyüş yapmayı planlıyorum.",
    "Hafta sonu sahilde vakit geçirmek harika olurdu.",
    "Bugün birkaç yeni tarif denedim, oldukça lezzetli oldu.",
    "Bu hafta işte oldukça yoğun geçiyor.",
    "Yarın için önemli bir randevum var.",
    "Bugün dışarıda yemek yemeyi planlıyorum.",
    "Evde temizlik yapmak için güzel bir gün.",
    "Bu sabah spora gitmek beni zorlayacak gibi görünüyor.",
    "Hafta sonu şehir dışına kısa bir kaçamak yapmayı düşünüyoruz.",
    "Sabahları kahvaltı yapmadan güne başlamam mümkün değil.",
    "Yeni bir hobi edinmek bana iyi gelebilir.",
    "Kitap okumak beni her zaman başka bir dünyaya götürüyor.",
    "Bugün çok verimli bir gün geçirdim.",
    "Yarın yeni bir projeye başlamayı planlıyorum.",
    "Bu hafta sonu ailecek vakit geçirmeyi planlıyoruz.",
    "Yeni bir film izledim, oldukça etkileyiciydi.",
    "Sabahları yürüyüş yapmak bana enerji veriyor.",
    "Dışarıda çok güzel bir hava var, belki biraz yürüyüş yaparım.",
    "Akşam yemeği için bir şeyler hazırlamam gerekiyor.",
    "Yarın arkadaşlarımla bir araya geleceğim, sabırsızlanıyorum.",
    "Bu hafta biraz daha fazla dinlenmeye ihtiyacım var.",
    "Bugün işlerimi erken bitirip biraz kitap okuyacağım.",
    "Yeni bir müzik albümü keşfetmek beni mutlu ediyor.",
    "Doğada vakit geçirmek beni her zaman sakinleştiriyor."
]

#### 7.8 LSTM 2

In [7]:
# metin temizleme ve preprocessing: tokenization, padding, label encoding

# tokenization 
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts) # metinler üzerindeki kelime frekanslarini ogren
total_words = len(tokenizer.word_index) + 1 # toplam kelime sayisi

# n-gram dizileri olustur ve padding uygula 
input_sequences = []
for text in texts: 
    # metinleri kelime dizilerine cevir
    token_list = tokenizer.texts_to_sequences([text])[0]

    # her metin icin n-gram dizisi olustur
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i + 1]
        input_sequences.append(n_gram_sequence)
# en uzun diziyi bulalım, tum dizileri ayni uzunluga getirelim
max_sequence_length = max(len(x) for x in input_sequences)

# dizilere diziyi bulalım, tum dizileri aynı uzunluga getirelim
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

# X ( girdi ) ve y ( hedef )
# 
X = input_sequences[:,:-1] # son kelimeyi almayacagiz
y = input_sequences[:,-1] # son kelimeyi alacagiz
y = tf.keras.utils.to_categorical(y, num_classes=total_words) # one-hot encoding

#### 7.9 LSTM 3

In [10]:
# LSTM modeli olustur, compile, train ve evaluate 
model = Sequential()

# embedding 
model.add(Embedding(total_words, 50, input_length = X.shape[1]))

# lstm 
model.add(LSTM(100, return_sequences=False))

# output
model.add(Dense(total_words, activation = "softmax"))

# model compile 
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

# model train
model.fit(X, y, epochs = 100, verbose = 1)

Epoch 1/100


19/19 [==============================] - 3s 9ms/step - loss: 5.7249 - accuracy: 0.0436
Epoch 2/100
19/19 [==============================] - 0s 8ms/step - loss: 5.5501 - accuracy: 0.0688
Epoch 3/100
19/19 [==============================] - 0s 9ms/step - loss: 5.3372 - accuracy: 0.0688
Epoch 4/100
19/19 [==============================] - 0s 8ms/step - loss: 5.2866 - accuracy: 0.0688
Epoch 5/100
19/19 [==============================] - 0s 8ms/step - loss: 5.2534 - accuracy: 0.0688
Epoch 6/100
19/19 [==============================] - 0s 8ms/step - loss: 5.2108 - accuracy: 0.0688
Epoch 7/100
19/19 [==============================] - 0s 8ms/step - loss: 5.1848 - accuracy: 0.0688
Epoch 8/100
19/19 [==============================] - 0s 8ms/step - loss: 5.1481 - accuracy: 0.0688
Epoch 9/100
19/19 [==============================] - 0s 9ms/step - loss: 5.0673 - accuracy: 0.0688
Epoch 10/100
19/19 [==============================] - 0s 8ms/step - loss: 4.9810 - accuracy: 0.0721
Epoch 1

In [17]:
# model prediction
def generate_text(seed_text, next_words):
    for _ in range(next_words):
        # girdi metnini sayisal verilere donustur
        token_list = tokenizer.texts_to_sequences([seed_text])[0]

        # padding 
        token_list = pad_sequences([token_list], maxlen = max_sequence_length-1, padding="pre")

        # prediction 
        predicted_probabilities = model.predict(token_list, verbose = 0)

        # en yuksek olasiliga sahip kelimenin indexini bul 
        predicted_word_index = np.argmax(predicted_probabilities, axis = 1)

        # tokenizer ile kelime index inden asil kelimeyi bul 
        predicted_word = tokenizer.index_word[predicted_word_index[0]]

        # tahmin edilen kelimeyi seed_text e ekleyelim
        seed_text = seed_text + " " + predicted_word
    return seed_text
seed_text = "sinemaya gitmek"
print(generate_text(seed_text, 6))

sinemaya gitmek her zaman keyifli bir aktivite götürüyor


### 7.10 Transformers Model (BERT)

#### 7.11 GPT

![ScreenS/7.11_Transformers.PNG](ScreenS/7.11_Transformers.PNG)


#### 7.12 LLAMA

- Transformers tabanlı 
- hafif ve verimli 
- daha kücük ve daha hızlı

![ScreenS/7.12_LLAMA.PNG](ScreenS/7.12_LLAMA.PNG)

![ScreenS/7.12_karsılastırma.PNG](ScreenS/7.12_karsılastırma.PNG)

#### 7.13 GPT ve LLAMA modelleri

In [ ]:
# import libraries
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# modelin tanimlanmasi 
model_name = "gpt2"

# tokenizer tanimla ve model olusturma
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

model = GPT2LMHeadModel.from_pretrained(model_name)

# metin uretimi icin gerekli olan baslagıc text i 
text = "Afternoon, "

# tokenization 
inputs = tokenizer.encode(text, return_tensors="pt")

# metin uretimi gerceklestirelim
outputs = model.generate(inputs, max_length = 20 )

# modelin urettigi tokenleri okunabilir hale getirmemiz lazım 
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True) # ozel tokenleri göstermemek icin "True" dedik

# uretilen metni print ettirelim 
print(generated_text)

c:\Users\mfurk\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mfurk\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP 

Afternoon,  I was sitting in the kitchen, and I was thinking about the next day


#### LLAMA

In [ ]:
# import libraries
from transformers import AutoModelForCausalLM, AutoTokenizer

# modelin tanimlanmasi 
model_name_llama = "huggyllama/llama-7b" #llama

# tokenizer tanimla ve model olusturma
tokenizer_llama = AutoTokenizer.from_pretrained(model_name_llama) #llama

model_llama = AutoModelForCausalLM.from_pretrained(model_name_llama) #llama

# metin uretimi icin gerekli olan baslagıc text i 
text = "Afternoon, "

# tokenization 
inputs_llama = tokenizer_llama.encode(text, return_tensors="pt") #llama

# metin uretimi gerceklestirelim
outputs_llama = model_llama.generate(inputs_llama, max_length = 20 ) #llama

# modelin urettigi tokenleri okunabilir hale getirmemiz lazım 
generated_text_llama = tokenizer_llama.decode(outputs[0], skip_special_tokens=True) #llama

# uretilen metni print ettirelim 
print(generated_text_llama) #llama